In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os

warnings.simplefilter('ignore')
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [ ]:
seed = 2020
v = 1

In [ ]:
# !pip install lightgbm

In [ ]:
df_feature = pd.read_pickle('data/feature{}.pkl'.format(v))

In [ ]:
df_feature.head()

In [ ]:
for f in df_feature.select_dtypes('object').columns:
    if f not in ['user']:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

In [ ]:
df_train = df_feature[df_feature.label.notna()].copy()
df_test = df_feature[df_feature.label.isna()].copy()

df_train.shape, df_test.shape

In [ ]:
ycol = 'label'
feature_names = list(
    filter(lambda x: x not in [ycol, 'user'], df_train.columns))

model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           num_leaves=32,
                           max_depth=6,
                           learning_rate=0.01,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.6,
                           reg_alpha=10,
                           reg_lambda=12,
                           random_state=seed,
                           is_unbalance=True,
                           metric='auc')

df_oof = df_train[['user', ycol]].copy()
df_oof['prob'] = 0
prediction = df_test[['user']]
prediction['prob'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val

    pred_test = lgb_model.predict_proba(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

In [ ]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

In [ ]:
auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('auc:', auc)

In [ ]:
# 0.737320648758525
# 0.7372858742331708

In [ ]:
os.makedirs('sub', exist_ok=True)
prediction.to_csv('sub/yizhifu_{}.csv'.format(auc), index=False)

In [ ]:
os.makedirs('prob', exist_ok=True)

prediction.to_csv('prob/sub_lgb{}.csv'.format(v), index=False)
df_oof[['user', 'prob', ycol]].to_csv('prob/oof_lgb{}.csv'.format(v), index=False)

In [ ]:
df_oof.head()

In [ ]:
prediction.head()